# Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")

# Load variables
import os
from dotenv import load_dotenv
load_dotenv()

# Snowpark Imports
from snowflake.snowpark.session import Session
from snowflake.ml.model.model_signature import FeatureSpec, DataType, ModelSignature

# Connect to Snowflake

In [2]:
snowflake_connection_cfg = {
    "ACCOUNT": os.getenv('SF_ACCOUNT'),
    "USER": os.getenv('SF_USER'),
    "ROLE": os.getenv('SF_ROLE'),
    "PASSWORD": os.getenv('SF_PASSWORD'),
    "DATABASE": os.getenv('SF_DATABASE'),
    "SCHEMA": os.getenv('SF_SCHEMA'),
    "WAREHOUSE": os.getenv('SF_WAREHOUSE')
}

# Creating Snowpark Session
session = Session.builder.configs(snowflake_connection_cfg).create()

print('Role:     ', session.get_current_role())
print('Warehouse:', session.get_current_warehouse())
print('Database: ', session.get_current_database())
print('Schema:   ', session.get_current_schema())

Role:      "ACCOUNTADMIN"
Warehouse: "COMPUTE_WH"
Database:  "MACHINE_LEARNING"
Schema:    "PUBLIC"


In [3]:
# Create some test data to work with
positive_reviews = [
    "Ich liebe dieses Produkt absolut! Es hat alle meine Erwartungen übertroffen und ich empfehle es jedem weiter.",
    "Das ist der beste Kauf, den ich dieses Jahr gemacht habe! Es funktioniert einwandfrei und der Kundenservice war fantastisch.",
    "Fünf Sterne! Das Produkt kam pünktlich an und funktioniert sogar besser als beschrieben. Könnte nicht glücklicher sein.",
    "Ich bin wirklich beeindruckt von der Qualität und Effizienz dieses Produkts. Es ist definitiv den Preis wert.",
    "Unglaubliche Ergebnisse! Ich habe sofort einen Unterschied bemerkt und es ist ein wesentlicher Teil meiner täglichen Routine geworden."
]

negative_reviews = [
    "Sehr enttäuscht von diesem Kauf. Es hat nach einer Woche aufgehört zu funktionieren und die Qualität ist minderwertig.",
    "Ich würde dieses Produkt nicht empfehlen. Es ist überteuert für das, was es bietet, und der Kundensupport ist nicht hilfreich.",
    "Ein Stern. Das Produkt kam beschädigt an und es zurückzusenden war ein Alptraum. Vollkommen unzufrieden.",
    "Das ist das schlechteste Produkt, das ich je benutzt habe. Es funktioniert nicht wie beworben und ist eine komplette Geldverschwendung.",
    "Ich bereue diesen Kauf. Es ist schlecht gemacht und hat keine meiner Erwartungen erfüllt. Eine totale Enttäuschung."
]

neutral_reviews = [
    "Das Produkt ist in Ordnung, funktioniert wie erwartet, aber nichts beeindruckendes.",
    "Es ist ein durchschnittliches Produkt, erledigt die Arbeit, sticht aber in keiner Weise hervor.",
    "Neutral bezüglich dieses Kaufs. Es hat seine Vor- und Nachteile, aber insgesamt ist es einfach in Ordnung.",
    "Das Produkt erfüllt die grundlegenden Anforderungen, ist jedoch nichts Außergewöhnliches.",
    "Es funktioniert ausreichend, allerdings gibt es bessere Optionen zu diesem Preis."
]



df = session.create_dataframe(positive_reviews+negative_reviews+neutral_reviews, schema=['TEXT'])
df.show(n=15, max_width=1000)

-------------------------------------------------------------------------------------------------------------------------------------------
|"TEXT"                                                                                                                                   |
-------------------------------------------------------------------------------------------------------------------------------------------
|Ich liebe dieses Produkt absolut! Es hat alle meine Erwartungen übertroffen und ich empfehle es jedem weiter.                            |
|Das ist der beste Kauf, den ich dieses Jahr gemacht habe! Es funktioniert einwandfrei und der Kundenservice war fantastisch.             |
|Fünf Sterne! Das Produkt kam pünktlich an und funktioniert sogar besser als beschrieben. Könnte nicht glücklicher sein.                  |
|Ich bin wirklich beeindruckt von der Qualität und Effizienz dieses Produkts. Es ist definitiv den Preis wert.                            |
|Unglaubliche Ergebn

# Register & Run Sentiment Model

In [4]:
# Get the model registry object
from snowflake.ml.registry import Registry
reg = Registry(
    session=session, 
    database_name=session.get_current_database(), 
    schema_name=session.get_current_schema()
    )

In [5]:
# Get the sentiment model from Huggingface
# Make sure it fits into a Snowflake warehouse and does not require GPUs
# Otherwise the model must deployed in Snowpark Container Services
from transformers import pipeline
pipe = pipeline("text-classification", model="lxyuan/distilbert-base-multilingual-cased-sentiments-student")

# use SOWH because model is big
session.use_warehouse('snowpark_opt_wh')

# Customize our model signature
model_sig = ModelSignature(
                  inputs=[
                      FeatureSpec(dtype=DataType.STRING, name='TEXT')
                      ],
                      outputs=[
                          FeatureSpec(dtype=DataType.STRING, name='label'),
                          FeatureSpec(dtype=DataType.DOUBLE, name='score')
                      ]
                  )

# Register the model to Snowflake
snow_model_custom = reg.log_model(
    pipe, 
    model_name='distilbert_base_multilingual_cased_sentiments_student_custom', 
    signatures={'predict':model_sig},
    conda_dependencies=['tokenizers','transformers']
    )

# Model signature
sentiment_values = snow_model_custom.run(df).cache_result()
sentiment_values.show(n=15, max_width=1000)

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"TEXT"                                                                                                                                   |"label"   |"score"              |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|Ein Stern. Das Produkt kam beschädigt an und es zurückzusenden war ein Alptraum. Vollkommen unzufrieden.                                 |negative  |0.8657243847846985   |
|Ich liebe dieses Produkt absolut! Es hat alle meine Erwartungen übertroffen und ich empfehle es jedem weiter.                            |positive  |0.95161372423172     |
|Fünf Sterne! Das Produkt kam pünktlich an und funktioniert sogar besser als beschrieben. Könnte nicht glücklicher sein.               